<a href="https://colab.research.google.com/github/skurakake/MMP_public/blob/main/Simple_CNN_c100_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [15]:
# cifar100 を使う
from tensorflow.keras.datasets import cifar100

(x_train, t_train), (x_test, t_test) = cifar100.load_data(label_mode='fine')


CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

num_category = 100 # クラス数

In [ ]:
# ダウンロードしたデータの一部をランダムに表示

print('x_train shape:', x_train.shape)
print('the number of train samples:', x_train.shape[0] )
print('the number of test samples:', x_test.shape[0] )
print(type(x_train[0]))

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)
print(type(x_test))
print(type(t_test[0]))


num_image = 20 # 表示数
rand_idx = np.random.randint(0, len(x_train), num_image)
plt.figure(figsize=(5, 5)) # 表示領域のサイズ
# plt figure set to 5inch x 5inch(500pixel x 500 pixel).

labels = CIFAR100_LABELS_LIST
for i in range(num_image):
    plt.subplot(4, 5, i + 1)
    plt.imshow(x_train[rand_idx[i]])
    plt.title(labels[t_train[rand_idx[i]][0]])
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)

plt.show() # 実行されたとき始めてウィンドウが立ち上がり、そのウィンドウに図が表示される
# colab では plt.show() がなくても、環境が自動でウィンドウをオープンしてくれるが、
# コマンドラインとかでは、これが必要

In [ ]:
# 設定
batch_size = 32 # ミニバッチサイズ
epochs = 20 # エポック数
num_class = 100 # クラス数

# 正解データを one-hot 表現へ変換
t_train_n = tf.keras.utils.to_categorical(t_train, num_class)
t_test_n = tf.keras.utils.to_categorical(t_test, num_class)
print("正解データ（学習）：", t_train_n[:2])
print("正解データ（テスト）：", t_test_n[:2])

モデルの構築

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Resizing, Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

model = Sequential()

model.add(Input(shape=x_train.shape[1:]))# .shape の2番目から最後までの要素 (32,32,3)
#model.add(Resizing(32, 32))
model.add(Rescaling(1./255))
model.add(Conv2D(32,(3,3), strides=1,  padding = 'same')) #padding = 'same' 入力と同じサイズになるまで０パディング
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3), strides=1,  padding = 'same')) #padding = 'same' 入力と同じサイズになるまで０パディング
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2))) #, strides=2))

model.add(Conv2D(64,(3,3), strides=1,  padding = 'same')) #padding = 'same' 入力と同じサイズになるまで０パディング
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3), strides=1,  padding = 'same')) #padding = 'same' 入力と同じサイズになるまで０パディング
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2))) #, strides=2))

model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.5))
model.add(Dense(num_class))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()


モデルを訓練する

In [ ]:
history = model.fit(x_train, t_train_n, batch_size=batch_size,
                    epochs=epochs, validation_data=(x_test, t_test_n))

In [ ]:
# 学習の経過をグラフで表示
import matplotlib.pyplot as plt
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
validation_loss = history.history['val_loss']
validation_accuracy = history.history['val_accuracy']

plt.plot(np.arange(len(train_loss)), train_loss, label='training_loss')
plt.plot(np.arange(len(validation_loss)), validation_loss, label='validation_loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

plt.plot(np.arange(len(train_accuracy)), train_accuracy, label='training_accuracy')
plt.plot(np.arange(len(validation_accuracy)), validation_accuracy, label='validation_accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

テストデータ全体に対して精度とlossを求める

In [ ]:
loss, accuracy = model.evaluate(x_test, t_test_n)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

学習済みのモデルを使って、テストデータの中からランダムに選択したデータに対して分類を行う。

In [ ]:
num2_image = 20
rand_idx2 = np.random.randint(0, len(x_test), num2_image)

y_rand = model.predict(x_test[rand_idx2]) # テストデータからランダムに画像を選択して、画像分類を実行
predicted_class = np.argmax(y_rand, axis=1) # 分類結果の中で最大値を取るインデックスを取得

plt.figure(figsize=(16,16)) # 画像の表示サイズ

for i in range(num2_image):
    plt.subplot(4, 5, i + 1)
    plt.imshow(x_test[rand_idx2[i]])
    label = cifar10_labels[predicted_class[i]]
    plt.title(label)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

plt.show()


学習済みのモデルをgoogle drive に保存

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
from keras.models import load_model

path = '/content/drive/MyDrive/Colab Notebooks/'

if not os.path.exists(path):
    os.makedirs(path)

#
model.save(path + "model_simple_CNN_c100_2.keras")